<a href="https://colab.research.google.com/github/Linux-Server/AI_Engineering/blob/main/llama3_8b_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
#@title Install Pre-requisite
!pip instal trl
!pip install unsloth

In [3]:
from unsloth import FastLanguageModel


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    load_in_4bit=True,
    max_seq_length=2048,
    dtype=None
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.3: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [4]:
## peft
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ


)

Unsloth 2025.10.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
## load dataset
from datasets import load_dataset

dataset = load_dataset("vicgalle/alpaca-gpt4", split="train")
dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-6ef3991c06080e(…):   0%|          | 0.00/48.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [6]:
dataset.column_names


['instruction', 'input', 'output', 'text']

In [7]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt="{instruction}[[\n This is the input : \n {input}]]",
    output_column_name="output",
    conversation_extension=2
)

Merging columns:   0%|          | 0/52002 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [8]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Give three tips for staying healthy.'},
  {'from': 'gpt',
   'value': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'},
  {'from': 'human', 'value': 'Describe what a monotheistic religion is.'},
  {'from': 'gpt',
   'value': 'A monotheis

In [9]:
from unsloth import standardize_sharegpt

dataset = standardize_sharegpt(dataset)


Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/52002 [00:00<?, ? examples/s]

In [10]:
dataset[0]

{'conversations': [{'content': 'Give three tips for staying healthy.',
   'role': 'user'},
  {'content': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
   'role': 'assistant'},
  {'content': 'Describe what a monotheistic religion is.', 'role': 'user'},
  {'content': 'A monotheistic re

In [11]:
tokenizer.chat_template

In [12]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [13]:
dataset[0]

{'conversations': [{'content': 'Give three tips for staying healthy.',
   'role': 'user'},
  {'content': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
   'role': 'assistant'},
  {'content': 'Describe what a monotheistic religion is.', 'role': 'user'},
  {'content': 'A monotheistic re

In [15]:
tokenizer.chat_template

"{{ bos_token }}{{ 'Below are some instructions that describe some tasks. Write responses that appropriately complete each request.' }}{% for message in messages %}{% if message['role'] == 'user' %}{{ '\n\n### Instruction:\n' + message['content'] }}{% elif message['role'] == 'assistant' %}{{ '\n\n### Response:\n' + message['content'] + '<|end_of_text|>' }}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '\n\n### Response:\n' }}{% endif %}"

In [17]:
from trl import SFTTrainer, SFTConfig

args = SFTConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_8bit",
    warmup_steps=5,
    learning_rate=2e-4,
    logging_steps=5,
    output_dir="./llama-8b",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    report_to="wandb",
    run_name="llama_run-1",
    max_steps=100,

   )

trainer = SFTTrainer(
    train_dataset=dataset,
    model=model,
    tokenizer=tokenizer,
    args=args,
    dataset_text_field="text",
    max_seq_length=2048
)

In [18]:
train_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 52,002 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sachin6624 (sachin6624-axomium-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.691300
10,1.330100
15,1.197600
20,1.129200
25,1.050800
30,1.082200
35,1.060800
40,1.034200
45,1.053600
50,1.096800


In [20]:
import torch
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
7.117 GB of memory reserved.


In [22]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{train_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(train_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

529.2658 seconds used for training.
8.82 minutes used for training.
Peak reserved memory = 7.117 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 32.115 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [23]:
train_stats

TrainOutput(global_step=100, training_loss=1.1024652528762817, metrics={'train_runtime': 529.2658, 'train_samples_per_second': 1.512, 'train_steps_per_second': 0.189, 'total_flos': 1.653624371847168e+16, 'train_loss': 1.1024652528762817, 'epoch': 0.015384023691396485})

In [35]:
## Inference
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

prompt = [{"role":"user", "content": "who is mohanlal?"}]
model_inputs = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
model_inputs

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

tokenized_inputs = tokenizer(model_inputs, return_tensors="pt").to(model.device)
tokenized_inputs


outs = model.generate(**tokenized_inputs, max_new_tokens=100, streamer=streamer)

# tokenizer.batch_decode(outs, skip_special_tokens=True)


Mohanlal is an Indian actor, producer and director who has worked in Malayalam, Tamil, Hindi, Telugu and Kannada films. He is considered one of the most popular and influential actors in Indian cinema and has received several awards including two National Film Awards, three Kerala State Film Awards and seven Filmfare Awards South. He is also a playback singer and has sung songs in Malayalam and Tamil films.


In [37]:
model.push_to_hub("sachin6624/llama3-8b-100steps-alpaca")
tokenizer.push_to_hub("sachin6624/llama3-8b-100steps-alpaca")

README.md:   0%|          | 0.00/577 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/sachin6624/llama3-8b-100steps-alpaca


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]